In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization
import os

# Tắt log rác của Tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# ===== 1. LOAD DỮ LIỆU =====
print("⏳ Đang load dữ liệu...")
try:
    X = np.load("X_data.npy")
    y = np.load("y_data.npy")
    classes = np.load("classes.npy")
except FileNotFoundError:
    print("❌ LỖI: Không tìm thấy file .npy! Hãy chắc chắn bạn đang ở đúng thư mục chứa file X_data.npy")
    exit()

print(f"✅ Dữ liệu input: {X.shape}") # (88, 130, 13)
print(f"✅ Số lớp cảm xúc: {len(classes)} {classes}")

# Chia tập Train (80%) và Test (20%)
# Vì dữ liệu ít (88 mẫu), ta để test_size nhỏ thôi để dành dữ liệu cho máy học
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ===== 2. THIẾT KẾ MODEL (CNN 1D) =====
input_shape = (X.shape[1], X.shape[2]) # (130, 13)
num_classes = y.shape[1]

model = Sequential()

# Block 1
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

# Block 2
model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

# Block 3
model.add(Conv1D(64, kernel_size=3, activation='relu')) # Giảm filter chút vì data ít
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())

# Fully Connected
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3)) # Dropout giúp giảm học vẹt
model.add(Dense(num_classes, activation='softmax')) # Output layer

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# ===== 3. TRAINING =====
print("\n🚀 BẮT ĐẦU TRAIN...")
# Batch size nhỏ (8 hoặc 16) vì tổng dữ liệu chỉ có 88 mẫu
history = model.fit(X_train, y_train, 
                    epochs=50, 
                    batch_size=8, 
                    validation_data=(X_test, y_test),
                    verbose=1)

# ===== 4. ĐÁNH GIÁ =====
print("\n📊 ĐÁNH GIÁ KẾT QUẢ TRÊN TẬP TEST:")
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"🎯 Độ chính xác (Accuracy): {acc*100:.2f}%")

# Báo cáo chi tiết
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print("\n--- CHI TIẾT TỪNG LỚP ---")
print(classification_report(y_true, y_pred_classes, target_names=classes))

# ===== 5. LƯU MODEL =====
model_name = "emotion_model.keras" # Đuôi mới của Keras là .keras (thay vì .h5)
model.save(model_name)
print(f"\n💾 Đã lưu model tại: {model_name}")
print("👉 Bây giờ bạn có thể dùng file này để dự đoán cảm xúc!")

e:\ONEXT\AI Research\venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.15) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


⏳ Đang load dữ liệu...
✅ Dữ liệu input: (88, 130, 13)
✅ Số lớp cảm xúc: 5 ['ANG' 'ANX' 'HAP' 'NEU' 'SAD']

🚀 BẮT ĐẦU TRAIN...
Epoch 1/50


e:\ONEXT\AI Research\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.3286 - loss: 2.4242 - val_accuracy: 0.2778 - val_loss: 8.0230
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6714 - loss: 0.9607 - val_accuracy: 0.2778 - val_loss: 5.0931
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7857 - loss: 0.5557 - val_accuracy: 0.2778 - val_loss: 3.7542
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9286 - loss: 0.3069 - val_accuracy: 0.3889 - val_loss: 2.5577
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9714 - loss: 0.1914 - val_accuracy: 0.3889 - val_loss: 3.2283
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9857 - loss: 0.1192 - val_accuracy: 0.3333 - val_loss: 3.5480
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9571 - loss: 0.1934 - val_accuracy: 0.3889 - val_loss: 3.0232
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9429 - loss: 0.1790 - val_accuracy: 0.5556 - val_loss: 2.3439
Epoch 9/50


e:\ONEXT\AI Research\venv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
e:\ONEXT\AI Research\venv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
e:\ONEXT\AI Research\venv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [7]:
import numpy as np
import librosa
import tensorflow as tf
import os

# ===== CẤU HÌNH =====
# Đường dẫn đến file audio bạn muốn thử
# Bạn có thể đổi tên file này thành file bất kỳ bạn muốn test
TEST_FILE = r"E:\KHMT\N4K2\DATN\data\raw_cuts_with_time_VNDC\0h01m55s_seg0022.wav" 

MODEL_PATH = "emotion_model.keras"
CLASSES_PATH = "classes.npy"

SAMPLE_RATE = 22050
DURATION = 3
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

# ===== HÀM XỬ LÝ (Phải giống hệt lúc train) =====
def extract_mfcc(file_path):
    try:
        signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
        if len(signal) > SAMPLES_PER_TRACK:
            signal = signal[:SAMPLES_PER_TRACK]
        else:
            padding = int(SAMPLES_PER_TRACK - len(signal))
            signal = np.pad(signal, (0, padding), mode='constant')
        mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13, n_fft=2048, hop_length=512)
        return mfcc.T
    except Exception as e:
        print(f"Lỗi đọc file: {e}")
        return None

# ===== CHƯƠNG TRÌNH DỰ ĐOÁN =====
# 1. Load Model và Nhãn
print("⏳ Đang load model...")
model = tf.keras.models.load_model(MODEL_PATH)
classes = np.load(CLASSES_PATH)
print(f"✅ Đã load model. Các nhãn AI biết: {classes}")

# 2. Xử lý file âm thanh
print(f"🎧 Đang nghe file: {os.path.basename(TEST_FILE)}")
mfcc = extract_mfcc(TEST_FILE)

if mfcc is not None:
    # Model cần input dạng (Số lượng, Thời gian, Đặc trưng) -> (1, 130, 13)
    mfcc = np.expand_dims(mfcc, axis=0) 

    # 3. Dự đoán
    prediction = model.predict(mfcc)
    predicted_index = np.argmax(prediction)
    predicted_label = classes[predicted_index]
    confidence = np.max(prediction) * 100 # Độ tin cậy

    print("\n" + "="*30)
    print(f"🤖 KẾT QUẢ DỰ ĐOÁN: {predicted_label}")
    print(f"🎯 Độ tin cậy: {confidence:.2f}%")
    print("="*30)
    
    # In ra xác suất của từng nhãn
    print("\nChi tiết xác suất:")
    for i, label in enumerate(classes):
        print(f"- {label}: {prediction[0][i]*100:.2f}%")

else:
    print("❌ Không xử lý được file âm thanh.")

⏳ Đang load model...
✅ Đã load model. Các nhãn AI biết: ['ANG' 'ANX' 'HAP' 'NEU' 'SAD']
🎧 Đang nghe file: 0h01m55s_seg0022.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step

🤖 KẾT QUẢ DỰ ĐOÁN: SAD
🎯 Độ tin cậy: 100.00%

Chi tiết xác suất:
- ANG: 0.00%
- ANX: 0.00%
- HAP: 0.00%
- NEU: 0.00%
- SAD: 100.00%
